# Example 1: Classification with Synthetic data


In [ ]:
import numpy as np
import networkx as nx
import pandas as pd
import scipy as sc

from pathlib import Path
import os

if not Path("datasets").exists():
    os.mkdir("datasets")
if not Path("results").exists():
    os.mkdir("results")

In [ ]:
# this cell is generating some synthetic graph dataset with node features.


# defining limits on number of nodes
n_min = 20
n_max = 50

# number of graphs
num_g = 100

# number of node features - in this example I will generate random node features that aren't useful for classifcation
n_nf = 3


# empty list of graphs and labels
graphs = []
labels = []
node_features = []


# setting limits on probability of edge existing for random graphs
p_min = 0.1
p_max = 0.5

# adding 50 random graphs (label 0)
for i in range(int(num_g / 2)):
    rand_n = np.random.randint(n_min, n_max)
    rand_p = np.random.randint(int(p_min * 100), int(p_max * 100)) / 100

    g = nx.fast_gnp_random_graph(rand_n, rand_p)
    node_feat_matrix = np.random.random((rand_n, n_nf))

    graphs.append(nx.to_numpy_array(g))
    node_features.append(node_feat_matrix)

    labels.append(0)

# setting limits on number of edges to add per node
m_min = 1
m_max = 5

# adding 50  powerlaw cluster graphs (label 1)
for i in range(int(num_g / 2)):
    rand_n = np.random.randint(n_min, n_max)
    rand_p = np.random.randint(int(p_min * 100), int(p_max * 100)) / 100
    rand_m = np.random.randint(m_min, m_max)

    g = nx.powerlaw_cluster_graph(rand_n, rand_m, rand_p)
    node_feat_matrix = np.random.random((rand_n, n_nf))

    graphs.append(nx.to_numpy_array(g))
    node_features.append(node_feat_matrix)

    labels.append(1)

# Load graphs into graph object

We now have three lists of length 100. The graphs list is composed of numpy arrays that represent the adjacency matrix of the graph. The node features list is composed of numpy arrays that contain the node information for each graph. The labels list is a list of integers that corresponds to the class label for each graph.

The next step is to take this data and convert it into an appropriate format for hcga.

In [ ]:
# converting this data into the format required for hcga

from hcga.graph import Graph, GraphCollection

# create graph collection object
g_c = GraphCollection()

# add graphs, node features and labels to the object
g_c.add_graph_list(graphs, node_features, labels)

In [ ]:
# perform some sanity checks

print("There are {} graphs".format(len(g_c.graphs)))
print("There are {} features per node".format(g_c.get_n_node_features()))

In [ ]:
# we can save this if we want to and run everything from the command line
from hcga.io import save_dataset

save_dataset(
    g_c,
    "custom_dataset_classification",
    folder="./datasets/custom_dataset_classification",
)

# Extracting features

We have now produced a pickle dataset of your own custom data. We can now run the feature extraction from the command line using the following commands:

hcga extract_features ./datasets/custom_dataset.pkl -m fast -n 4 -sl advanced --timeout 10 


Alternatively,we could import the Hcga class and run the feature extraction and analysis from within the notebook. We will do this below.

In [ ]:
# import hcga object
from hcga.hcga import Hcga

# define an object
h = Hcga()

# load previously saved dataset
h.load_data(
    "./datasets/custom_dataset_classification/custom_dataset_classification.pkl"
)

In [ ]:
# extracting all features here
h.extract(mode="fast", n_workers=4, timeout=20)

# saving all features into a pickle
h.save_features("./results/custom_dataset_classification/all_features.pkl")

# Analysis - classification example

In [ ]:
# load the saved features

h.load_features("./results/custom_dataset_classification/all_features.pkl")

In [ ]:
# implement a classification analyse of the features

h.analyse_features(
    feature_file="./results/custom_dataset_classification/all_features.pkl",
    results_folder="./results/custom_dataset_classification",
)